In [1]:
import string
from nltk.corpus import stopwords
import nltk 
import pandas as pd
import numpy as np
import re
from collections import Counter
import csv
from nltk import stem
import difflib

In [2]:
#nltk.download_shell() 

In [3]:
from nltk.corpus import stopwords

# Dictionary

In [4]:
abb = pd.read_csv("abb_new.csv")
abb =abb.drop_duplicates()
abb =abb.dropna()
print(abb.head())

                              Name  Form
0               Above Ground Level   AGL
1           Actual Time Of Arrival   ATA
2         Actual Time Of Departure   ATD
3                     Aeronautical  AERO
4  Aeronautical Information Manual   AIM


In [5]:
print(len(abb))
print(len(abb['Form'].unique()))

465
456


In [6]:
abb['Form'].value_counts().head(10)
#some rows have the same column value

CAT     2
N       2
FSS     2
ATA     2
TCAS    2
Z       2
AWOS    2
EST     2
RWY     2
OJT     1
Name: Form, dtype: int64

In [7]:
array = ['FSS','EST','CAT','AWOS','RWY','Z','ATA','TCAS','N']
abb.loc[abb['Form'].isin(array)]

,Name,Form
1,Actual Time Of Arrival,ATA
13,Air Traffic Area,ATA
40,Automated Surface Observation Sys (Or Weather ...,AWOS
46,Automatic Weather Observing/Reporting System,AWOS
65,Category,CAT
79,Clear Air Turbulence,CAT
97,Coordinated Universal Time,Z
127,Eastern Standard Time,EST
139,"Estimate, Estimation",EST
178,Flight Service Station,FSS


In [8]:
#Unify some values
abb.loc[1,'Name'] = 'Actual Time Of Arrival/Air Traffic Area'
abb.loc[13,'Name'] = 'Actual Time Of Arrival/Air Traffic Area'
abb.loc[40,'Name'] = 'Automatic Weather Observing System'
abb.loc[46,'Name'] = 'Automatic Weather Observing System'
abb.loc[65,'Name'] ='Category/Clear Air Turbulence'
abb.loc[79,'Name'] ='Category/Clear Air Turbulence'
abb.loc[468,'Name'] = 'Coordinated Universal Time'
abb.loc[127,'Name'] = 'Eastern Standard Time/Estimate'
abb.loc[139,'Name'] = 'Eastern Standard Time/Estimate'
abb.loc[179,'Name'] = 'Flight Service Station'
abb.loc[465,'Name'] = 'North'
abb.loc[387,'Name'] = 'Runway(s)'
abb.loc[467,'Name'] = 'Traffic Alert and Collision Avoidance'

In [9]:
abb =abb.drop_duplicates()
abb =abb.dropna()
abb['Form'].value_counts().head(5)

N       2
TCAS    2
RWY     2
Z       2
APCH    1
Name: Form, dtype: int64

In [10]:
#Remove rows that contain numeric values
indexNames = abb[(abb['Form'].str.contains('0')) | (abb['Form'].str.contains('1'))|
                 (abb['Form'].str.contains('2')) | (abb['Form'].str.contains('3'))|
                 (abb['Form'].str.contains('4')) | (abb['Form'].str.contains('5'))|
                 (abb['Form'].str.contains('6')) | (abb['Form'].str.contains('7'))|
                 (abb['Form'].str.contains('8')) | (abb['Form'].str.contains('9'))].index
abb.drop(indexNames , inplace=True)

In [11]:
print(len(abb)),print(len(abb['Form'].unique()))
abb.head()

458
454


,Name,Form
0,Above Ground Level,AGL
1,Actual Time Of Arrival/Air Traffic Area,ATA
2,Actual Time Of Departure,ATD
3,Aeronautical,AERO
4,Aeronautical Information Manual,AIM


In [12]:
#Create the dictionary
dict_abb = {}
for i in range(len(abb)):   
    word = ' '+abb.iloc[i,1]+' '
    value = ' '+abb.iloc[i,0]+' '
    dict_abb[word] = value
#Standard 2400' Hi-Intensity Approach Lighting System With Sequenced Flashers (Cat I Config), ALSF-1
#Standard 2400' Hi-Intensity Approach Lighting System With Sequenced Flashers (Cat Ii Config), ALSF-2
dict_abb = {k.lower(): v.lower() for k, v in dict_abb.items()}

# ASRS

In [13]:
df = pd.read_csv('asrs_v1_1112.csv')
df.columns[-16:-10]

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (15,19,20,28,33,44,51,53,54,55,56,66,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['report 1', 'report 1.1', 'report 2', 'report 2.1', 'report 1.2',
       'unnamed: 96'],
      dtype='object')

In [14]:
df = df.head(5000)
re1 = df['report 1']

In [15]:
def text_process1(mess):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ' '+''.join(nopunc).lower()+' '
    return nopunc

In [16]:
re1 =re1.apply(text_process1)

In [17]:
re1[0]

' today flying at 9000 ft near msv on an ifr flt plan i saw 2 parachuters drop past within 100 ft the first at my 11 oclock pos btwn 50 ft and 100 ft away saw that one but didnt comprehend what it was went by too fast i thought it was some kind of collapsed balloon at first then another at my 2303 oclock pos less than 50 ft off my wing just after that atc told me to turn to an immediate 160 deg hdg i had been on a 90 deg hdg i told them what i had just seen and he said yeah there was a jump acft above me only mention of para jumping in the fss briefing was nw of wilkesbarre and atc hadnt said a peep before the incident i was in and out of cumulus at that alt and had only been out of a cloud for about 30 seconds before the jumpers passed through my alt too fast for me to attempt any evasive action their parachutes were not deployed but a small parachute or an unreleased parachute trailed each by a few ft the jumpers were close enough to see clrly that they were wearing helmets with viso

In [18]:
#replace abbreviation with words
for i in range(len(re1)):
    for k, v in dict_abb.items():
        re1[i] = re1[i].replace(k, v)
re1_new = re1.copy()

In [19]:
for i in range(len(re1)):
    for k, v in dict_abb.items():
        re1[i] = re1[i].replace(k, v)

In [20]:
re1_new[0]

' today flying at 9000 feet/foot near msv on an instrument flight rules flight plan i saw 2 parachuters drop past within 100 feet/foot the first at my 11 oclock position between 50 feet/foot and 100 feet/foot away saw that one but didnt comprehend what it was went by too fast i thought it was some kind of collapsed balloon at first then another at my 2303 oclock position less than 50 feet/foot off my wing just after that air traffic control told me to turn to an immediate 160 degree heading i had been on a 90 degree heading i told them what i had just seen and he said yeah there was a jump aircraft above me only mention of para jumping in the flight service station briefing was nw of wilkesbarre and air traffic control hadnt said a peep before the incident i was in and out of cumulus at that altitude and had only been out of a cloud for about 30 seconds before the jumpers passed through my altitude too fast for me to attempt any evasive action their parachutes were not deployed but a s

In [21]:
#remove blanks
for i in range(len(re1_new)):
    re1_new[i] = re1_new[i][1:-1]
re1_new[0]

'today flying at 9000 feet/foot near msv on an instrument flight rules flight plan i saw 2 parachuters drop past within 100 feet/foot the first at my 11 oclock position between 50 feet/foot and 100 feet/foot away saw that one but didnt comprehend what it was went by too fast i thought it was some kind of collapsed balloon at first then another at my 2303 oclock position less than 50 feet/foot off my wing just after that air traffic control told me to turn to an immediate 160 degree heading i had been on a 90 degree heading i told them what i had just seen and he said yeah there was a jump aircraft above me only mention of para jumping in the flight service station briefing was nw of wilkesbarre and air traffic control hadnt said a peep before the incident i was in and out of cumulus at that altitude and had only been out of a cloud for about 30 seconds before the jumpers passed through my altitude too fast for me to attempt any evasive action their parachutes were not deployed but a sm

In [22]:
#add new report to previous csv file
#result = pd.concat([df, re1_new], axis=1, sort=False)
#result.to_csv('asrs_v1_0105.csv')

# Word Count

In [23]:
#Combine all lists to one list
re1_word=[]
for i in range(len(re1_new)):
    re1_new[i] = re1_new[i].split(' ')
    re1_word = re1_word +re1_new[i]

In [28]:
nltk.download('wordnet')
wnl = stem.WordNetLemmatizer()
porter = stem.porter.PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lukmanbun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
def word_count(x):
    #Remove stop words from the list
    stop_words = stopwords.words('english')
    x =[word for word in x if word not in stop_words]
    #Stemming
    #re1_word = [stem(word) for word in re1_word]
    x = [wnl.lemmatize(word) for word in x]
    #remove words contain number
    x = [word for word in x if all(digit not in word for digit in "1234567890")]
    #remove blank
    return [word for word in x if word != '']

In [117]:
re1_word_remove = word_count(re1_word)

In [32]:
frequency

,Word,Count
0,the,88002
1,to,50212
2,i,41989
3,and,41009
4,was,28996
...,...,...
23853,001002,1
23854,001,1
23855,0001,1
23856,this,1


In [118]:
#word count
counts = Counter(re1_word_remove)
frequency =[(l,k) for k,l in sorted([(j,i) for i,j in counts.items()], reverse=True)]
frequency = pd.DataFrame(frequency)
frequency.columns = ['Word','Count']

In [119]:
#add frequency column
frequency['Percentage'] = round(100 * frequency['Count'] / frequency['Count'].sum(),2)

In [120]:
frequency[0:10]

,Word,Count,Percentage
0,aircraft,11130,1.55
1,flight,10761,1.50
2,runway(s),8407,1.17
3,feet/foot,7730,1.08
4,traffic,6050,0.84
5,approach,5911,0.82
6,time,4531,0.63
7,landing,4420,0.62
8,tower,4324,0.60
9,rule,4151,0.58


In [121]:
frequency.to_csv('word_count_report1.csv')

# Extracing Regular Words

In [136]:
# import two dictionaries to remove regular words
dict_regular = pd.read_csv("dictionary.csv")
dict_engmix_df = pd.read_csv("engmix.csv", encoding='latin1')

In [152]:
dict_engmix = list(dict_engmix_df.iloc[:, 0])
len(dict_engmix)

84098

In [46]:
# store the first column with words 
dict_words = dict_regular.iloc[:, 0]
len(dict_words)

54554

In [54]:
# remove duplicated words and make them lowercase
dict_words = list(dict.fromkeys(dict_words))
dict_words = [x.lower() for x in dict_words]
len(dict_words)

34240

In [141]:
# remove function
def remove_words(original_dict, dict_):
    filter_bool = []
    for word in original_dict["Word"]:
        filter_bool.append(word not in dict_)
        
    new_dict = original_dict[filter_bool]
    return new_dict

In [142]:
# use the dict_words to remove
dict_temp = remove_words(frequency, dict_words)

In [156]:
# use the dict_engmix to remove
dict_abnormal = remove_words(dict_temp, dict_engmix)
len(dict_abnormal)

6628

In [158]:
dict_abnormal.head(10)

,Word,Count,Percentage
2,runway(s),8407,1.17
3,feet/foot,7730,1.08
42,clred,1989,0.28
59,taxiway,1581,0.22
67,rptr,1426,0.20
77,degs,1356,0.19
150,meridiem,848,0.12
151,omni-directional,847,0.12
153,zzz,836,0.12
161,didnt,803,0.11


In [157]:
dict_abnormal.to_csv("abnormal_abbre_counts.csv")